In [49]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

N=50
T=60
P=7
R=0.8
M=5
F=0.005

In [50]:
def GetData(file):

    data = pd.read_csv(file)
    data = data[["datadate","tic","adjcp"]]
    return data

In [51]:
def PartitionData(data):
    DateToIndex={}
    IndexToDate={}
    SmallerData=[]
    i=0
    for date in data["datadate"].unique(): 
        temp = data[data["datadate"]==date].copy()
        SmallerData.append(temp)
        DateToIndex[int(date)] = i
        IndexToDate[i] = int(date)
        i+=1

    return DateToIndex, IndexToDate, SmallerData

In [52]:
temp = PartitionData(GetData("DATA.csv"))
DateToIndex=temp[0]
IndexToDate=temp[1]
SmallerData=temp[2]

In [63]:
# def MomentumBasedPriority(today):
#     k=DateToIndex[int(today)]
#     j=k-N
#     sum = np.zeros(30,dtype = float)
#     open = []
#     close = []
#     for i in range(N):
#         data = SmallerData[j]
#         t=0
#         for cp in data["adjcp"]:
#             sum[t]+=float(cp)
#             if(i==0):
#                 open.append(int(cp))
#             elif(i==49):
#                 close.append(int(cp))
#             t+=1
#         j+=1

#     ans = np.zeros((30,2), dtype = [('first', float),('second', int)])
    
#     for i in range(30):
#         ans[i][0] = (close[i]-open[i])/sum[i]
#         ans[i][1]=i
    
#     sorted_indices = np.argsort(ans[:, 0])[::-1]
#     ans = ans[sorted_indices]

#     return ans

def GetMomentumBasedPriority(SmallerData, DayToIndex, today):
    today = int(today)
    k = DayToIndex[today]
    j = k-N

    sum = [0.0]*30
    sum = np.array(sum)
    first=[]

    data = SmallerData[j]
    for value in data["adjcp"]:
        first.append(value)
    
    for it in range(N):
        data = SmallerData[j]
        i=0
        for value in data["adjcp"]:
            sum[i]+=float(value)
            i+=1
        j+=1

    data = SmallerData[k]
    i=0
    ans = np.zeros((30, 2))

    for value in data["adjcp"]:
        div = sum[i]
        temp = (value - first[i]) / div
        ans[i][0] = temp
        ans[i][1] = i
        i+=1
    
    sorted_indices = np.argsort(ans[:, 0])[::-1]
    ans = ans[sorted_indices]
    return ans

In [96]:
class PortFolio:
    initial = 0.0
    current = 0.0
    holdings = np.zeros(30, dtype = float)
    latest = np.zeros(30, dtype = float)
    possible = []

    def __init__(self, ini):
        self.initial = ini
        self.current = ini
        

    def SellStock(self, stock):
        self.current+=self.holdings[stock]*self.latest[stock]
        self.holdings[stock] = 0.0

    def BuyStock(self, stock, number):
        self.current-= number*self.latest[stock]
        self.holdings[stock] = number
        
    def CalculateNetWorth(self):
        ans = 0.0
        for i in range(30):
            ans+=self.holdings[i]*self.latest[i]
        
        return ans+self.current

    def ChangePricesTo(self, prices):
        self.latest = prices

    def RebalancePortfolio(self, ans, t):
        
        for i in range(15):
            if len(me.possible)+1>M:
                break
            me.possible.append((ans[i][1][1],ans[29-i][1][1]))
        for pair in me.possible:
            if t==0:
                me.SellStock(pair[1])
                me.BuyStock(pair[0], 1)
            elif t==1:
                me.SellStock(pair[0])
                me.BuyStock(pair[1], 1)
        

In [103]:
me = PortFolio(10000)
NetWorths = []
it = 51
for i in range(1):
    dur=0
    today = IndexToDate[it]
    ans = MomentumBasedPriority(today)
    t=0
    while dur<T:
        dur+=P
        if dur>T:
            break
        me.RebalancePortfolio(ans,t)    
        NetWorths.append(me.CalculateNetWorth())
        it+=P
        prices = []
        for val in SmallerData[it]["adjcp"]:
            prices.append(val)
        me.ChangePricesTo(prices)
        t=1-t

In [104]:
print(NetWorths)

[10000.0, 10246.476785714283, 10248.807088744588, 10257.412445887448, 10261.102565899451, 10277.474423042308, 10282.914993099312, 10285.95035024217]
